In [5]:
import pymysql
import pandas as pd
import numpy as np
import db_conn
import copy

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Data Load

In [6]:
player_stat = pd.DataFrame(db_conn.select_query('select * from player_stat'))
stat_code = pd.DataFrame(db_conn.select_query('select * from stat_code'))
match_results = pd.DataFrame(db_conn.select_query('select * from wc_match'))
player_info = pd.DataFrame(db_conn.select_query('select * from player'))

In [7]:
match_results['winning_flag'] = match_results.apply(lambda x: 'draw' if x.home_team_score == x.away_team_score else x.home_team_id if x.home_team_score > x.away_team_score else x.away_team_id, axis=1)
match_results.head()

,away_team_id,away_team_score,home_team_id,home_team_score,id,match_date,match_location,winning_flag
0,2,0,1,5,1,14 JUN 2018 18:00,Moscow / Luzhniki Stadium / RUS,1
1,4,1,3,0,2,15 JUN 2018 17:00,Ekaterinburg / Ekaterinburg Arena / RUS,4
2,6,3,5,3,3,15 JUN 2018 21:00,Sochi / Fisht Olympic Stadium / RUS,draw
3,8,1,7,0,4,15 JUN 2018 18:00,Saint Petersburg /,8
4,10,1,9,2,5,16 JUN 2018 13:00,Kazan / Kazan Arena / RUS,9


In [8]:
player_stat.head()

,half,id,match_id,player_id,stat_type,stat_value
0,total,1,1,1,POS,Goalkeeper
1,2,2,1,1,SAVE,2
2,1,3,1,1,DAT,1
3,1,4,1,1,CC,2
4,1,5,1,1,CA,2


### Extract stat list

In [9]:
u_stat = player_stat[['half', 'stat_type']].drop_duplicates().groupby('stat_type')['half'].unique().reset_index()

### Embeding
#### Excluding Stats with String values

In [10]:
new_stat = copy.deepcopy(player_stat[(player_stat.stat_type != 'POS') & (player_stat.stat_type != 'SUB')])

#### Stat values numerizing

In [11]:
def numerize(x):
    if x.__contains__("'"):
        ms = x.split("'")
        if ms.__len__() == 1 or ms[1] == '':
            return int(ms[0]) * 60
        else:
            return int(ms[0]) * 60 + int(ms[1].split('"')[0])
    elif x.__contains__('"'):
        return int(x.replace('"', ''))
    elif x.__contains__('%'):
        return float(x.replace('%', '')) / 100
    elif x.__contains__(','):
        return int(x.replace(',', ''))
    else:
        return x

new_stat.stat_value = new_stat.stat_value.apply(lambda x: numerize(x))
new_stat.stat_value = new_stat.stat_value.apply(pd.to_numeric)

In [12]:
embeded_ps = pd.DataFrame(columns = ['match_id', 'player_id'] + u_stat.stat_type.values.tolist())
embeded_ps[['match_id', 'player_id']] = player_stat[['match_id', 'player_id']].drop_duplicates().sort_values(['match_id', 'player_id']).reset_index(drop=True)

#### Create new dataframe by match id and player id

In [13]:
embeded_ps.head()

,match_id,player_id,2YR,ASSIST,CA,CC,DAT,DC,DCIP,DCNIP,DPA,DZONE1,DZONE2,DZONE3,DZONE4,DZONE5,FOUL,Long_PA,Long_PC,Medium_PA,Medium_PC,OFF,POS,RC,SAVE,SCORE,SHOT,SPA,SPRINT,SRAT,SRPA,SUB,Short_PA,Short_PC,TA3RD,TG,TNG,TOH,TPA,TS,TSCORE,TSCORE_OG,TSCORE_PEN,TSL,TSNL,TTP,TZONE1,TZONE2,TZONE3,TZONE4,TZONE5,Total_PA,Total_PC,Total_PP,YC
0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
def get_stat_value(match_id, player_id):
    result = []
    for s in u_stat.iterrows():
        stat_values = new_stat[(new_stat.match_id == match_id) & (new_stat.player_id == player_id) & (new_stat.stat_type == s[1].stat_type)]
        if stat_values.__len__() == 0:
            result.append(0)
        elif 'total' in stat_values.half.values.tolist():
            result.append(stat_values[stat_values.half == 'total'].stat_value.sum())
        else:
            result.append(stat_values.stat_value.sum())

    return result

In [15]:
embeded_ps = embeded_ps.apply(lambda x: pd.Series([x.match_id, x.player_id] + get_stat_value(x.match_id, x.player_id),index=embeded_ps.columns), axis=1)

#### Filtering Stats with String values

In [16]:
pos_stat = copy.deepcopy(player_stat[(player_stat.stat_type == 'POS') | (player_stat.stat_type == 'SUB')])

#### Add POS and SUB

In [17]:
def get_position_and_sub(x):
    pos = ''
    sub = ''
#     pos_stat[(pos_stat.match_id == x.match_id) & (pos_stat.player_id == x.player_id) & (pos_stat.stat_type == 'POS')].stat_value.values[0]
    if pos_stat[(pos_stat.match_id == x.match_id) & (pos_stat.player_id == x.player_id) & (pos_stat.stat_type == 'POS')].__len__() > 0:
        pos = pos_stat[(pos_stat.match_id == x.match_id) & (pos_stat.player_id == x.player_id) & (pos_stat.stat_type == 'POS')].stat_value.values[0]
        
    if pos_stat[(pos_stat.match_id == x.match_id) & (pos_stat.player_id == x.player_id) & (pos_stat.stat_type == 'SUB')].__len__() > 0:
        sub = pos_stat[(pos_stat.match_id == x.match_id) & (pos_stat.player_id == x.player_id) & (pos_stat.stat_type == 'POS')].stat_value.values[0]
        
    x.POS = pos
    x.SUB = sub
        
    return x

In [18]:
embeded_ps = embeded_ps.apply(lambda x: get_position_and_sub(x), axis=1)

### Save to file

In [19]:
embeded_ps.to_csv('./embeded_ps2.csv', sep='\t')

In [20]:
new_stat[(new_stat.match_id == 1) & (new_stat.player_id == 1) & (new_stat.stat_type == 'CA')].stat_value.sum()

3.0

In [21]:
embeded_ps.head(30)

,match_id,player_id,2YR,ASSIST,CA,CC,DAT,DC,DCIP,DCNIP,DPA,DZONE1,DZONE2,DZONE3,DZONE4,DZONE5,FOUL,Long_PA,Long_PC,Medium_PA,Medium_PC,OFF,POS,RC,SAVE,SCORE,SHOT,SPA,SPRINT,SRAT,SRPA,SUB,Short_PA,Short_PC,TA3RD,TG,TNG,TOH,TPA,TS,TSCORE,TSCORE_OG,TSCORE_PEN,TSL,TSNL,TTP,TZONE1,TZONE2,TZONE3,TZONE4,TZONE5,Total_PA,Total_PC,Total_PP,YC
0,1.0,1.0,0.0,0.0,3.0,3.0,1.0,4027.0,1818.0,1252.0,0.0,2955.0,919.0,137.0,16.0,0.0,0.0,25.0,10.0,9.0,9.0,0.0,Goalkeeper,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,,0.0,0.0,0.00,0.0,0.0,0.00,0.00,14.83,0.0,0.0,0.0,0.0,0.0,5808.0,0.94,0.06,0.01,0.00,0.00,34.0,19.0,0.56,0.0
1,1.0,2.0,0.0,0.0,5.0,4.0,2.0,7997.0,2836.0,3301.0,0.0,3830.0,2593.0,877.0,500.0,197.0,0.0,8.0,7.0,39.0,36.0,0.0,Defender,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,,10.0,9.0,0.05,1.0,2.0,0.08,0.02,27.90,0.0,0.0,0.0,0.0,1.0,5808.0,0.79,0.16,0.04,0.01,0.01,57.0,52.0,0.91,0.0
2,1.0,3.0,0.0,0.0,8.0,7.0,1.0,9040.0,3494.0,3465.0,0.0,4145.0,2942.0,1049.0,689.0,215.0,1.0,13.0,7.0,30.0,27.0,0.0,Defender,0.0,0.0,0.0,0.0,0.0,47.0,0.0,0.0,,10.0,9.0,0.04,0.0,0.0,0.11,0.03,29.48,0.0,0.0,0.0,0.0,1.0,5808.0,0.76,0.17,0.04,0.02,0.01,53.0,43.0,0.81,0.0
3,1.0,4.0,0.0,0.0,2.0,1.0,6.0,9951.0,3942.0,3774.0,0.0,3005.0,4358.0,1349.0,786.0,453.0,0.0,4.0,3.0,36.0,30.0,0.0,Defender,0.0,0.0,0.0,0.0,0.0,47.0,3.0,0.0,,24.0,22.0,0.06,0.0,2.0,0.24,0.00,28.80,0.0,0.0,0.0,0.0,0.0,5808.0,0.64,0.27,0.06,0.02,0.01,64.0,55.0,0.86,0.0
4,1.0,5.0,0.0,0.0,3.0,2.0,6.0,11323.0,4986.0,3969.0,2.0,3511.0,4462.0,2131.0,936.0,283.0,1.0,10.0,8.0,48.0,46.0,1.0,Midfield,0.0,0.0,0.0,0.0,0.0,52.0,1.0,0.0,,19.0,17.0,0.13,0.0,0.0,0.38,0.01,28.37,0.0,0.0,0.0,2.0,0.0,5808.0,0.63,0.26,0.07,0.03,0.01,77.0,71.0,0.92,0.0
5,1.0,6.0,0.0,0.0,0.0,0.0,3.0,8587.0,3765.0,2998.0,0.0,2412.0,3581.0,1731.0,657.0,206.0,0.0,1.0,1.0,16.0,14.0,0.0,Midfield,0.0,0.0,0.0,0.0,0.0,34.0,1.0,0.0,Midfield,13.0,12.0,0.21,1.0,3.0,0.52,0.04,30.38,0.0,0.0,0.0,1.0,2.0,4430.0,0.61,0.28,0.08,0.02,0.01,30.0,27.0,0.90,0.0
6,1.0,7.0,0.0,0.0,0.0,0.0,1.0,8854.0,4109.0,2808.0,0.0,3334.0,3400.0,1444.0,462.0,214.0,2.0,0.0,0.0,6.0,6.0,0.0,Forward,0.0,0.0,0.0,1.0,0.0,28.0,0.0,0.0,Forward,1.0,1.0,0.29,1.0,0.0,0.72,0.06,30.60,0.0,0.0,0.0,0.0,0.0,5164.0,0.69,0.22,0.06,0.02,0.01,7.0,7.0,1.00,0.0
7,1.0,8.0,0.0,0.0,8.0,6.0,1.0,10509.0,4027.0,3726.0,0.0,3691.0,3961.0,1632.0,810.0,415.0,1.0,4.0,2.0,29.0,22.0,0.0,Defender,0.0,0.0,0.0,2.0,0.0,50.0,1.0,0.0,,16.0,16.0,0.12,1.0,2.0,0.29,0.01,32.40,0.0,0.0,0.0,0.0,0.0,5808.0,0.68,0.23,0.06,0.02,0.01,49.0,40.0,0.82,0.0
8,1.0,9.0,0.0,0.0,0.0,0.0,3.0,8137.0,3529.0,3003.0,0.0,2292.0,3844.0,1404.0,491.0,106.0,0.0,7.0,5.0,23.0,22.0,0.0,Midfield,0.0,0.0,0.0,0.0,0.0,21.0,2.0,0.0,Midfield,21.0,20.0,0.11,0.0,1.0,0.28,0.03,27.90,0.0,0.0,0.0,0.0,0.0,3932.0,0.58,0.32,0.08,0.02,0.00,51.0,47.0,0.92,0.0
9,1.0,10.0,0.0,0.0,1.0,1.0,1.0,10874.0,4456.0,3810.0,0.0,3888.0,4383.0,2029.0,496.0,78.0,3.0,2.0,1.0,25.0,24.0,0.0,Midfield,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,,13.0,12.0,0.13,0.0,2.0,0.40,0.02,27.00,0.0,0.0,0.0,0.0,1.0,5808.0,0.66,0.25,0.08,0.01,0.00,40.0,37.0,0.92,1.0


In [22]:
match_id = 1
player_id = 1

team_id = player_info[player_info.id == player_id].team_id.values[0]
match_results[match_results.id == match_id].winning_flag == team_id

0    False
Name: winning_flag, dtype: bool

In [23]:
embeded_ps_w_label = copy.deepcopy(embeded_ps)

In [24]:
def get_team_id(player_id):
    if player_info[player_info.id == player_id].__len__() == 0:
        return 'check player info'
    else:
        return player_info[player_info.id == player_id].team_id.values[0]

In [25]:
embeded_ps_w_label['team_id'] = embeded_ps_w_label.player_id.apply(lambda x: get_team_id(x))

In [26]:
match_results.head()

,away_team_id,away_team_score,home_team_id,home_team_score,id,match_date,match_location,winning_flag
0,2,0,1,5,1,14 JUN 2018 18:00,Moscow / Luzhniki Stadium / RUS,1
1,4,1,3,0,2,15 JUN 2018 17:00,Ekaterinburg / Ekaterinburg Arena / RUS,4
2,6,3,5,3,3,15 JUN 2018 21:00,Sochi / Fisht Olympic Stadium / RUS,draw
3,8,1,7,0,4,15 JUN 2018 18:00,Saint Petersburg /,8
4,10,1,9,2,5,16 JUN 2018 13:00,Kazan / Kazan Arena / RUS,9


In [27]:
def get_winning_flag(match_id, team_id):
    if team_id == 'check player info':
        return 'check player info'

    if match_results[match_results.id == match_id].winning_flag.values[0] == 'draw':
        return 'draw'
    elif match_results[match_results.id == match_id].winning_flag.values[0] == team_id:
        return 'win'
    else:
        return 'lose'

In [37]:
def get_team_score(match_id, team_id):
    if team_id == 'check player info':
        return -1

    m_result = match_results[match_results.id == match_id]
    
    if m_result.home_team_id.values[0] == team_id:
        return m_result.home_team_score.values[0]
    elif m_result.away_team_id.values[0] == team_id:
        return m_result.away_team_score.values[0]
    else:
        return -1

In [38]:
embeded_ps_w_label['winning_flag'] = embeded_ps_w_label.apply(lambda x: get_winning_flag(x.match_id, x.team_id), axis=1)
embeded_ps_w_label['team_score'] = embeded_ps_w_label.apply(lambda x: get_team_score(x.match_id, x.team_id), axis=1)

In [39]:
embeded_ps_w_label.head(30)

,match_id,player_id,2YR,ASSIST,CA,CC,DAT,DC,DCIP,DCNIP,DPA,DZONE1,DZONE2,DZONE3,DZONE4,DZONE5,FOUL,Long_PA,Long_PC,Medium_PA,Medium_PC,OFF,POS,RC,SAVE,SCORE,SHOT,SPA,SPRINT,SRAT,SRPA,SUB,Short_PA,Short_PC,TA3RD,TG,TNG,TOH,TPA,TS,TSCORE,TSCORE_OG,TSCORE_PEN,TSL,TSNL,TTP,TZONE1,TZONE2,TZONE3,TZONE4,TZONE5,Total_PA,Total_PC,Total_PP,YC,team_id,winning_flag,team_score
0,1.0,1.0,0.0,0.0,3.0,3.0,1.0,4027.0,1818.0,1252.0,0.0,2955.0,919.0,137.0,16.0,0.0,0.0,25.0,10.0,9.0,9.0,0.0,Goalkeeper,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,,0.0,0.0,0.00,0.0,0.0,0.00,0.00,14.83,0.0,0.0,0.0,0.0,0.0,5808.0,0.94,0.06,0.01,0.00,0.00,34.0,19.0,0.56,0.0,2,lose,0
1,1.0,2.0,0.0,0.0,5.0,4.0,2.0,7997.0,2836.0,3301.0,0.0,3830.0,2593.0,877.0,500.0,197.0,0.0,8.0,7.0,39.0,36.0,0.0,Defender,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,,10.0,9.0,0.05,1.0,2.0,0.08,0.02,27.90,0.0,0.0,0.0,0.0,1.0,5808.0,0.79,0.16,0.04,0.01,0.01,57.0,52.0,0.91,0.0,2,lose,0
2,1.0,3.0,0.0,0.0,8.0,7.0,1.0,9040.0,3494.0,3465.0,0.0,4145.0,2942.0,1049.0,689.0,215.0,1.0,13.0,7.0,30.0,27.0,0.0,Defender,0.0,0.0,0.0,0.0,0.0,47.0,0.0,0.0,,10.0,9.0,0.04,0.0,0.0,0.11,0.03,29.48,0.0,0.0,0.0,0.0,1.0,5808.0,0.76,0.17,0.04,0.02,0.01,53.0,43.0,0.81,0.0,2,lose,0
3,1.0,4.0,0.0,0.0,2.0,1.0,6.0,9951.0,3942.0,3774.0,0.0,3005.0,4358.0,1349.0,786.0,453.0,0.0,4.0,3.0,36.0,30.0,0.0,Defender,0.0,0.0,0.0,0.0,0.0,47.0,3.0,0.0,,24.0,22.0,0.06,0.0,2.0,0.24,0.00,28.80,0.0,0.0,0.0,0.0,0.0,5808.0,0.64,0.27,0.06,0.02,0.01,64.0,55.0,0.86,0.0,2,lose,0
4,1.0,5.0,0.0,0.0,3.0,2.0,6.0,11323.0,4986.0,3969.0,2.0,3511.0,4462.0,2131.0,936.0,283.0,1.0,10.0,8.0,48.0,46.0,1.0,Midfield,0.0,0.0,0.0,0.0,0.0,52.0,1.0,0.0,,19.0,17.0,0.13,0.0,0.0,0.38,0.01,28.37,0.0,0.0,0.0,2.0,0.0,5808.0,0.63,0.26,0.07,0.03,0.01,77.0,71.0,0.92,0.0,2,lose,0
5,1.0,6.0,0.0,0.0,0.0,0.0,3.0,8587.0,3765.0,2998.0,0.0,2412.0,3581.0,1731.0,657.0,206.0,0.0,1.0,1.0,16.0,14.0,0.0,Midfield,0.0,0.0,0.0,0.0,0.0,34.0,1.0,0.0,Midfield,13.0,12.0,0.21,1.0,3.0,0.52,0.04,30.38,0.0,0.0,0.0,1.0,2.0,4430.0,0.61,0.28,0.08,0.02,0.01,30.0,27.0,0.90,0.0,2,lose,0
6,1.0,7.0,0.0,0.0,0.0,0.0,1.0,8854.0,4109.0,2808.0,0.0,3334.0,3400.0,1444.0,462.0,214.0,2.0,0.0,0.0,6.0,6.0,0.0,Forward,0.0,0.0,0.0,1.0,0.0,28.0,0.0,0.0,Forward,1.0,1.0,0.29,1.0,0.0,0.72,0.06,30.60,0.0,0.0,0.0,0.0,0.0,5164.0,0.69,0.22,0.06,0.02,0.01,7.0,7.0,1.00,0.0,2,lose,0
7,1.0,8.0,0.0,0.0,8.0,6.0,1.0,10509.0,4027.0,3726.0,0.0,3691.0,3961.0,1632.0,810.0,415.0,1.0,4.0,2.0,29.0,22.0,0.0,Defender,0.0,0.0,0.0,2.0,0.0,50.0,1.0,0.0,,16.0,16.0,0.12,1.0,2.0,0.29,0.01,32.40,0.0,0.0,0.0,0.0,0.0,5808.0,0.68,0.23,0.06,0.02,0.01,49.0,40.0,0.82,0.0,2,lose,0
8,1.0,9.0,0.0,0.0,0.0,0.0,3.0,8137.0,3529.0,3003.0,0.0,2292.0,3844.0,1404.0,491.0,106.0,0.0,7.0,5.0,23.0,22.0,0.0,Midfield,0.0,0.0,0.0,0.0,0.0,21.0,2.0,0.0,Midfield,21.0,20.0,0.11,0.0,1.0,0.28,0.03,27.90,0.0,0.0,0.0,0.0,0.0,3932.0,0.58,0.32,0.08,0.02,0.00,51.0,47.0,0.92,0.0,2,lose,0
9,1.0,10.0,0.0,0.0,1.0,1.0,1.0,10874.0,4456.0,3810.0,0.0,3888.0,4383.0,2029.0,496.0,78.0,3.0,2.0,1.0,25.0,24.0,0.0,Midfield,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,,13.0,12.0,0.13,0.0,2.0,0.40,0.02,27.00,0.0,0.0,0.0,0.0,1.0,5808.0,0.66,0.25,0.08,0.01,0.00,40.0,37.0,0.92,1.0,2,lose,0


-----------------------------

### Using winning flag For label

In [42]:
embeded_ps_w_label[['match_id', 'player_id', '2YR', 'ASSIST', 'CA', 'CC', 'DAT', 'DC',
       'DCIP', 'DCNIP', 'DPA', 'DZONE1', 'DZONE2', 'DZONE3', 'DZONE4',
       'DZONE5', 'FOUL', 'Long_PA', 'Long_PC', 'Medium_PA', 'Medium_PC', 'OFF',
       'POS', 'RC', 'SAVE', 'SCORE', 'SHOT', 'SPA', 'SPRINT', 'SRAT', 'SRPA',
       'SUB', 'Short_PA', 'Short_PC', 'TA3RD', 'TG', 'TNG', 'TOH', 'TPA', 'TS',
       'TSCORE', 'TSCORE_OG', 'TSCORE_PEN', 'TSL', 'TSNL', 'TTP', 'TZONE1',
       'TZONE2', 'TZONE3', 'TZONE4', 'TZONE5', 'Total_PA', 'Total_PC',
       'Total_PP', 'YC', 'team_id', 'winning_flag']].to_csv('embeded_with_winning_flag_label.tsv', sep='\t', index=False)

### Using team score For label

In [43]:
embeded_ps_w_label[['match_id', 'player_id', '2YR', 'ASSIST', 'CA', 'CC', 'DAT', 'DC',
       'DCIP', 'DCNIP', 'DPA', 'DZONE1', 'DZONE2', 'DZONE3', 'DZONE4',
       'DZONE5', 'FOUL', 'Long_PA', 'Long_PC', 'Medium_PA', 'Medium_PC', 'OFF',
       'POS', 'RC', 'SAVE', 'SCORE', 'SHOT', 'SPA', 'SPRINT', 'SRAT', 'SRPA',
       'SUB', 'Short_PA', 'Short_PC', 'TA3RD', 'TG', 'TNG', 'TOH', 'TPA', 'TS',
       'TSCORE', 'TSCORE_OG', 'TSCORE_PEN', 'TSL', 'TSNL', 'TTP', 'TZONE1',
       'TZONE2', 'TZONE3', 'TZONE4', 'TZONE5', 'Total_PA', 'Total_PC',
       'Total_PP', 'YC', 'team_id', 'team_score']].to_csv('embeded_with_team_score_label.tsv', sep='\t', index=False)

--------

In [109]:
embeded_ps_w_label.winning_flag.unique()

array(['lose', 'win', 'draw', 'check player info'], dtype=object)

In [110]:
embeded_ps_w_label[embeded_ps_w_label.winning_flag == 'check player info']

,match_id,player_id,2YR,ASSIST,CA,CC,DAT,DC,DCIP,DCNIP,DPA,DZONE1,DZONE2,DZONE3,DZONE4,DZONE5,FOUL,Long_PA,Long_PC,Medium_PA,Medium_PC,OFF,POS,RC,SAVE,SCORE,SHOT,SPA,SPRINT,SRAT,SRPA,SUB,Short_PA,Short_PC,TA3RD,TG,TNG,TOH,TPA,TS,TSCORE,TSCORE_OG,TSCORE_PEN,TSL,TSNL,TTP,TZONE1,TZONE2,TZONE3,TZONE4,TZONE5,Total_PA,Total_PC,Total_PP,YC,team_id,winning_flag
947,34.0,523.0,0.0,0.0,0.0,0.0,2.0,10108.0,0.0,0.0,0.0,3350.0,3962.0,1614.0,776.0,406.0,1.0,0.0,0.0,0.0,0.0,0.0,Midfield,0.0,0.0,0.0,3.0,0.0,44.0,0.0,0.0,Midfield,0.0,0.0,0.0,0.0,1.0,0.0,0.0,31.97,0.0,0.0,0.0,0.0,2.0,5239.0,0.64,0.26,0.07,0.02,0.01,0.0,0.0,0.0,0.0,check player info,check player info


In [47]:
db_conn.select_query('select * from wc_match where id = 34')

[{'away_team_id': 3,
  'away_team_score': 1,
  'home_team_id': 2,
  'home_team_score': 2,
  'id': 34,
  'match_date': '25 JUN 2018 17:00',
  'match_location': 'Volgograd / Volgograd Arena / RUS'}]

In [49]:
db_conn.select_query('select * from team where id in (2, 3)')

[{'country': 'Saudi Arabia', 'id': 2}, {'country': 'Egypt', 'id': 3}]

In [53]:
db_conn.select_query('select * from player where name like "%TREZ%"')

[{'birth': '01/10/94',
  'club': 'Kasimpasa SK (TUR)',
  'height': 181,
  'id': 38,
  'name': 'TREZEGUET',
  'num': 21,
  'position': 'MF',
  'team_id': 3}]